# Trino + Iceberg Demo

This notebook demonstrates SQL queries on Iceberg tables using Trino.

## Setup

The notebook comes pre-installed with:
- `trino` - Trino Python client
- `pandas` - Data manipulation
- `ipython-sql` - SQL magic commands
- `matplotlib`, `seaborn`, `plotly` - Visualization
- `pyarrow` - Arrow/Parquet support

## Method 1: Using Trino Python Client

In [ ]:
from trino.dbapi import connect
import pandas as pd

# Connect to Trino
conn = connect(
    host="trino.sovereign-dp.svc.cluster.local",
    port=8080,
    user="jupyter",
    catalog="iceberg",
    schema="default"
)

cursor = conn.cursor()
print("✅ Connected to Trino!")

In [ ]:
# Show available catalogs
cursor.execute("SHOW CATALOGS")
catalogs = cursor.fetchall()
print("Available catalogs:")
for catalog in catalogs:
    print(f"  - {catalog[0]}")

In [ ]:
# Show schemas in Iceberg catalog
cursor.execute("SHOW SCHEMAS FROM iceberg")
schemas = cursor.fetchall()
print("Iceberg schemas:")
for schema in schemas:
    print(f"  - {schema[0]}")

## Create a Demo Schema and Table

In [ ]:
# Create a demo schema
cursor.execute("CREATE SCHEMA IF NOT EXISTS iceberg.demo")
print("✅ Schema 'demo' created")

In [ ]:
# Create a sample table
cursor.execute("""
CREATE TABLE IF NOT EXISTS iceberg.demo.sales (
    id BIGINT,
    product VARCHAR,
    quantity INT,
    price DECIMAL(10,2),
    sale_date DATE
)
""")
print("✅ Table 'sales' created")

In [ ]:
# Insert sample data
cursor.execute("""
INSERT INTO iceberg.demo.sales VALUES
    (1, 'Laptop', 2, 999.99, DATE '2024-01-15'),
    (2, 'Mouse', 5, 29.99, DATE '2024-01-16'),
    (3, 'Keyboard', 3, 79.99, DATE '2024-01-16'),
    (4, 'Monitor', 1, 299.99, DATE '2024-01-17'),
    (5, 'Laptop', 1, 999.99, DATE '2024-01-18')
""")
print("✅ Sample data inserted")

## Query and Analyze Data

In [ ]:
# Query data into a pandas DataFrame
df = pd.read_sql("SELECT * FROM iceberg.demo.sales", conn)
df

In [ ]:
# Calculate total revenue by product
revenue_query = """
SELECT 
    product,
    SUM(quantity) as total_quantity,
    SUM(quantity * price) as total_revenue
FROM iceberg.demo.sales
GROUP BY product
ORDER BY total_revenue DESC
"""

revenue_df = pd.read_sql(revenue_query, conn)
revenue_df

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

# Bar chart of revenue by product
fig, ax = plt.subplots(figsize=(10, 6))
revenue_df.plot(kind='bar', x='product', y='total_revenue', ax=ax, legend=False)
ax.set_title('Total Revenue by Product')
ax.set_xlabel('Product')
ax.set_ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Method 2: Using SQL Magic

In [ ]:
# Load SQL magic extension
%load_ext sql

In [ ]:
# Connect using SQL magic (user@host format)
%sql trino://jupyter@trino.sovereign-dp.svc.cluster.local:8080/iceberg/demo

In [ ]:
%%sql
SELECT * FROM sales LIMIT 5

In [ ]:
%%sql
SELECT 
    sale_date,
    COUNT(*) as num_sales,
    SUM(quantity * price) as daily_revenue
FROM sales
GROUP BY sale_date
ORDER BY sale_date

## Iceberg Time Travel

Apache Iceberg supports time travel - querying data as it existed at a specific point in time.

In [ ]:
# Show table snapshots
cursor.execute("""
SELECT 
    snapshot_id,
    committed_at,
    operation
FROM iceberg.demo."sales$snapshots"
ORDER BY committed_at DESC
""")

snapshots = cursor.fetchall()
print("Table snapshots:")
for snap in snapshots:
    print(f"  ID: {snap[0]}, Time: {snap[1]}, Op: {snap[2]}")

## Clean Up

In [ ]:
# Drop table (optional)
# cursor.execute("DROP TABLE IF EXISTS iceberg.demo.sales")
# print("✅ Table dropped")